## Müllabfuhr


[Aufgabe](muellabfuhr.pdf) - [Lösungshinweise](muellabfuhrL.pdf)

Beispiel 1:

<img src='bild2.png' width='400'>

### Kürzeste Wege mit Floyd

Der Floyd-Algorithmus liefert uns kürzeste Wege zwischen zwei beliebigen Knoten.
Die beiden beiden berechneten Matrizen D (Distanzmatrix) und P (Vorgängerknoten des kürzesten Wegs)
speichern wir als globale Variablen. Der Floyd-Algorithmus erwartet als Eingabe eine Adjazenzmatrix


#### Adjazenzmatrix erstellen

In [5]:
inf = float('inf')
nr = 1
f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
anzV, anzE = [int(x) for x in f.readline().split()]

GM = [[inf]*anzV for _ in range(anzV)]
V = set()
E = set()
for i in range(anzE):
    von, bis, km = [int(x) for x in f.readline().split()]
    GM[von][bis]=km
    GM[bis][von]=km
    V.add(von)
    V.add(bis)
    
    if von < bis:
        E.add((von,bis))
    else:
        E.add((bis,von))
for i in range(anzV):
    GM[i][i]=0

In [6]:
def floyd(c):
    n = len(c)
    d = [[0]*n for j in range(n)]   # Matrix D
    p = [[0]*n for j in range(n)]   # Matrix P
    for i in range(n):
        for j in range(n):
            d[i][j] = c[i][j]
            p[i][j] = i

    for k in range(n):
        for i in range(n):
            for j in range(n):
                tmp = d[i][k] + d[k][j]
                if tmp < d[i][j]:
                    d[i][j] = tmp
                    p[i][j] = p[k][j]
    return d, p

def getPath(p, i, j):
    if i == j: return (i,)
    return getPath(p,i,p[i][j]) + (j,)

def printMatrix(a):
    for i in range(len(a)):
        for j in range(len(a)):
            if a[i][j] == inf:
                print("{:4}".format("."),end='')
            else:
                print("{:<4}".format(a[i][j]),end='')
        print()
    print()
    
D, P = floyd(GM)

In [7]:
printMatrix(D)

0   2   9   2   6   4   1   2   
2   0   9   2   8   4   1   2   
9   9   0   7   14  10  8   9   
2   2   7   0   7   3   1   2   
6   8   14  7   0   5   7   7   
4   4   10  3   5   0   3   2   
1   1   8   1   7   3   0   1   
2   2   9   2   7   2   1   0   



Jetzt können wir die Distanz und die kürzesten Wege zwischen zwei beliebigen Knoten berechnen.

In [8]:
# die Matrizen D und P werden als globale Variablen vorausgesetzt.
def dist(v1, v2):
    '''
    returns: kürzeste Distanz zwischen v1 und v2  
    '''
    return D[v1][v2]

def sPath(v1, v2):
    '''
    returns: einen kürzesten Weg zwischen v1 und v2
    '''
    return getPath(P,v1,v2)

In [9]:
print(sPath(0,2))
print(dist(0,2))

(0, 6, 3, 2)
9


### Kreis, der alle Kanten durchläuft.

Ein Kreis, der alle Kanten nur einmal durchläuft, ist ein Eulerkreis. Der existiert genau dann, wenn wir alle Knoten einen geraden Grad haben. Das können wir bei unseren Graphen nicht erwarten. Wir verbinden die Knoten, die ungeraden Grad haben paarweise miteinander (möglichst billige Strecken), dann konstruieren wir den Eulerkreis. Dazu benötigen wir eine Modellierung des Graphen als Adjazenzliste. Allerdings brauchen wir hier die Gewichte nicht berücksichtigen (wir wollen ja alle Kanten durchlaufen, egal was sie für ein Gewicht haben). Nur beim matching der ungeraden Kanten benötigen wir die Information über die kürzesten Wege. Die holen wir uns aus dem Ergebnis des Floyd-Algorithmus. 

#### Adjazenzliste erstellen

In [11]:
nr = 1
f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
anzV, anzE = [int(x) for x in f.readline().split()]
G = {x:set() for x in range(anzV)}
for _ in range(anzE):
    v1, v2, _ = f.readline().split()
    v1, v2 = int(v1), int(v2)
    G[v1].add(v2)
    G[v2].add(v1)

In [12]:
G

{0: {4, 5, 6},
 1: {3, 6},
 2: {3},
 3: {1, 2, 4, 5, 6},
 4: {0, 3, 5, 7},
 5: {0, 3, 4, 7},
 6: {0, 1, 3, 7},
 7: {4, 5, 6}}

#### Knoten mit ungeradem Grad bestimmen

In [13]:
U = []
for v in G:
    if len(G[v]) % 2 != 0:
        U.append(v)
print(U)
print(len(U))

[0, 2, 3, 7]
4


#### Greedy Matching für die Knoten mit ungeradem Grad

```
Setze result = []
Solange U noch nicht leer:
    hole ein Element v1 aus U
    wähle aus dasjenige Element v2, das zu v1 den geringsten Abstand hat
    Füge (v1,v2) in die Liste result ein.
    Lösche v1, v2 aus U
```

In [14]:
def match(U):
    U1 = U.copy()          # U soll unverändert bleiben
    result = []
    while U1:
        v = U1.pop()       # v1 ist damit automatisch aus U gelöscht
        best_val = inf
        best = None
        for w in U1:
            if dist(v,w) < best_val:
                best_val = dist(v,w)
                best = w
        result.append((v,best))
        U1.remove(best)
    return result     

In [15]:
match(U) 

[(7, 0), (3, 2)]

Kosten des Matchings:


In [16]:
def matchCost(U):
    cost = 0
    for v,w in match(U):
        print(dist(v,w))
        cost += dist(v,w)
    return cost

print(f'Kosten des Matchings {matchCost(U)}')

2
7
Kosten des Matchings 9


#### Einfügen der Metakanten in den Graphen
Es kann sein, dass eine Kante eingefügt werden soll zwischen zwei Knoten, die schon eine Kante haben.
Unsere Implementation des Graphen unterstützt keine doppelten Kanten. Deswegen fügen wir auch einen Metaknoten zwischendrin ein. Wir nummerieren in ab 10000 aufwärts um ihn als Metaknoten erkennen zu können.

In [17]:
def addMeta(U):
    global hilf
    for e in match(U):
        v1, v2 = e
        G[hilf] = {v1,v2}
        G[v1].add(hilf)
        G[v2].add(hilf)
        hilf+=1


In [18]:
hilf = 10000
addMeta(U)
G

{0: {4, 5, 6, 10000},
 1: {3, 6},
 2: {3, 10001},
 3: {1, 2, 4, 5, 6, 10001},
 4: {0, 3, 5, 7},
 5: {0, 3, 4, 7},
 6: {0, 1, 3, 7},
 7: {4, 5, 6, 10000},
 10000: {0, 7},
 10001: {2, 3}}

#### Eulerkreis erstellen
Jetzt haben alle Knoten einen geraden Grad und wir können den Eulerkreis erstellen.

In [19]:
import sys
from copy import deepcopy

GE = deepcopy(G)
degrees = {x:len(GE[x]) for x in GE} 

def dfs(v):

    while degrees[v] != 0:
        next = list(GE[v])[0]    # die erste noch vorhandene Kante
        GE[v].remove(next)       # wird aus dem Graphen gelöscht
        GE[next].remove(v)       # von beiden Seiten
        degrees[v]-=1            # der Grad der Knoten v und next
        degrees[next]-=1         # wird erniedrigt
        dfs(next)
    path.append(v) 
    
path = []
dfs(0)                         # beim undirected eulercircle unnötig
print(path)    

[0, 6, 7, 5, 4, 3, 10001, 2, 3, 6, 1, 3, 5, 0, 4, 7, 10000, 0]


In [20]:
len(path)

18

Links und rechts der 10000er Zahlen stehen die Knoten, zwischen die wir den kürzesten Weg zueinander schreiben müssen.

In [21]:
def eliminateMeta(path):
    result = []
    for i in range(1,len(path)-1):
        if path[i] < 10000:
            result.append(path[i])
        else:
            v1, v2 = path[i-1],path[i+1]
            for v in sPath(v1,v2)[1:-1]:
                result.append(v)
    return result

In [22]:
path = eliminateMeta(path)
print(path)
print(len(path))

[6, 7, 5, 4, 3, 2, 3, 6, 1, 3, 5, 0, 4, 7, 6]
15


#### Den Eulerkreis aufteilen

In [23]:
def abschnitt(p,i,j):
    '''
    p: Pfad als Tuple oder Liste
    i, j: Indizes des Pfads, i <= j
    returns: Pfad (Liste) bestehend aus kürzestem Weg von 0 nach p[i], dann weiter auf p bis p[j], dann den
      kürzesten Weg p[j] zurück nach 0 
    '''
    return sPath(0,p[i])+tuple(p[i+1:j])+sPath(p[j],0)

def pathCost(p):
    '''
    p: Pfad
    returns: Summe der Kantenkosten entlang des Pfades
    '''
    pcost = 0                                  
    for k in range(len(p)-1):
        pcost += GM[p[k]][p[k+1]] 
    return pcost

def aufteilung(p,n):
    '''
    Aufteilung von Pfad p in 5 Abschnitte, wobei jeder Abschnitt <= n kostet
    returns: Liste mit Abschnitten der Indizes von Pfad p. Wenn n zu klein ist,
    decken die Abschnitte nicht den ganzen Pfad ab.
    z.B: [(0, 6), (6, 7), (7, 9), (9, 12), (12, 14)]
    '''
    tmp = []
    i = 0
    j = 0
    for k in range(5):
        while j < len(p) and (pathCost(abschnitt(p,i,j))) <= n:
            j+=1
        tmp.append((i,j-1))
        i=j-1
        j=i+1
    return tmp

def aufteilungOK(p,n):
    '''
    p: Pfad
    returns True, wenn die Aufteilung mit n den ganzen Pfad überdeckt
    '''
    au = aufteilung(path,n)
    lastIndex = au[-1][1]
    return lastIndex == len(path)-1

def printPath(p):
    '''
    printed den Pfad p mit seinen Kosten
    '''
    cost = 0
    for i in range(len(p)-1):
        
        cost += D[p[i]][p[i+1]]
    print(cost)

In [24]:
p = [0, 5, 7, 6, 3, 6, 7, 4, 5, 3, 4, 0, 6, 1, 3, 2, 3, 6, 0]
abschnitt(p,6,10)

(0, 6, 7, 4, 5, 3, 4, 0)

In [25]:
sPath(3,0)

(3, 6, 0)

#### Binäre Suche, um die beste Aufteilung des Eulerkreises zu finden

In [27]:
def besteAufteilung(path):

    mincost = 0
    maxcost = pathCost(abschnitt(path,0,len(path)-1))

    best = maxcost
    versuch = maxcost//2

    while True:

        if aufteilungOK(path,versuch):
            best = versuch
            versuch = versuch//2
        else:
            versuch1 = (versuch+best) // 2
            if versuch1 == versuch: 
                 break
            versuch = versuch1
            
    return best, aufteilung(path,best)

 

In [28]:
best, teile =   besteAufteilung(path)
print('Länge des Pfads',len(path))
print('Beste Aufteilung',teile)
print('Maximale Pfadkosten bei bester Aufteilung:', best)

Länge des Pfads 15
Beste Aufteilung [(0, 4), (4, 7), (7, 10), (10, 11), (11, 14)]
Maximale Pfadkosten bei bester Aufteilung: 18


#### Fahrplan für die 5 Müllabfuhren

In [29]:
print(f'Fahrplan für Beispiel {nr}:')
maxcost = 0
for teil in teile:
    p = abschnitt(path,teil[0],teil[1])
    cost = pathCost(p)
    if cost > maxcost:
        maxcost = cost
    print(f'Kosten: {cost} - {p}')
print(f'Maximale Kosten einer Fahrt: {maxcost}')

Fahrplan für Beispiel 1:
Kosten: 18 - (0, 6, 7, 5, 4, 3, 6, 0)
Kosten: 18 - (0, 6, 3, 2, 3, 6, 0)
Kosten: 18 - (0, 6, 1, 3, 5, 7, 6, 0)
Kosten: 10 - (0, 6, 7, 5, 0)
Kosten: 16 - (0, 4, 7, 6, 0)
Maximale Kosten einer Fahrt: 18


### Das vollständige Programm

In [30]:
from copy import deepcopy
def floyd(c):
    n = len(c)
    d = [[0]*n for j in range(n)]   # Matrix D
    p = [[0]*n for j in range(n)]   # Matrix P
    for i in range(n):
        for j in range(n):
            d[i][j] = c[i][j]
            p[i][j] = i

    for k in range(n):
        for i in range(n):
            for j in range(n):
                tmp = d[i][k] + d[k][j]
                if tmp < d[i][j]:
                    d[i][j] = tmp
                    p[i][j] = p[k][j]
    return d, p

def getPath(p, i, j):
    if i == j: return (i,)
    return getPath(p,i,p[i][j]) + (j,)

def printMatrix(a):
    for i in range(len(a)):
        for j in range(len(a)):
            if a[i][j] == inf:
                print("{:4}".format("."),end='')
            else:
                print("{:<4}".format(a[i][j]),end='')
        print()
    print()
    
def dist(v1, v2):
    '''
    returns: kürzeste Distanz zwischen v1 und v2  
    '''
    return D[v1][v2]

def sPath(v1, v2):
    '''
    returns: einen kürzesten Weg zwischen v1 und v2
    '''
    return getPath(P,v1,v2)

def match(U):
    U1 = U.copy()          # U soll unverändert bleiben
    result = []
    while U1:
        v = U1.pop()       # v1 ist damit automatisch aus U gelöscht
        best_val = inf
        best = None
        for w in U1:
            if dist(v,w) < best_val:
                best_val = dist(v,w)
                best = w
        result.append((v,best))
        U1.remove(best)
    return result  

def addMeta(U):
    global hilf
    for e in match(U):
        v1, v2 = e
        G[hilf] = {v1,v2}
        G[v1].add(hilf)
        G[v2].add(hilf)
        hilf+=1

def dfs(v):
    while degrees[v] != 0:
        next = list(GE[v])[0]    # die erste noch vorhandene Kante
        GE[v].remove(next)       # wird aus dem Graphen gelöscht
        GE[next].remove(v)       # von beiden Seiten
        degrees[v]-=1            # der Grad der Knoten v und next
        degrees[next]-=1         # wird erniedrigt
        dfs(next)
    path.append(v) 

def eliminateMeta(path):
    result = []
    for i in range(1,len(path)-1):
        if path[i] < 10000:
            result.append(path[i])
        else:
            v1, v2 = path[i-1],path[i+1]
            for v in sPath(v1,v2)[1:-1]:
                result.append(v)
    return result

def abschnitt(p,i,j):
    '''
    p: Pfad als Tuple oder Liste
    i, j: Indizes des Pfads, i <= j
    returns: Pfad (Liste) bestehend aus kürzestem Weg von 0 nach p[i], dann weiter auf p bis p[j], dann den
      kürzesten Weg p[j] zurück nach 0 
    '''
    return sPath(0,p[i])+tuple(p[i+1:j])+sPath(p[j],0)

def pathCost(p):
    '''
    p: Pfad
    returns: Summe der Kantenkosten entlang des Pfades
    '''
    pcost = 0                                  
    for k in range(len(p)-1):
        pcost += GM[p[k]][p[k+1]] 
    return pcost

def aufteilung(p,n):
    '''
    Aufteilung von Pfad p in 5 Abschnitte, wobei jeder Abschnitt <= n kostet
    returns: Liste mit Abschnitten der Indizes von Pfad p. Wenn n zu klein ist,
    decken die Abschnitte nicht den ganzen Pfad ab.
    z.B: [(0, 6), (6, 7), (7, 9), (9, 12), (12, 14)]
    '''
    tmp = []
    i = 0
    j = 0
    for k in range(5):
        while j < len(p) and (pathCost(abschnitt(p,i,j))) <= n:
            j+=1
        tmp.append((i,j-1))
        i=j-1
        j=i+1
    return tmp

def aufteilungOK(p,n):
    '''
    p: Pfad
    returns True, wenn die Aufteilung mit n den ganzen Pfad überdeckt
    '''
    au = aufteilung(path,n)
    lastIndex = au[-1][1]
    return lastIndex == len(path)-1

def printPath(p):
    '''
    printed den Pfad p mit seinen Kosten
    '''
    cost = 0
    for i in range(len(p)-1):
        
        cost += D[p[i]][p[i+1]]
    print(cost)
    
def besteAufteilung(path):

    mincost = 0
    maxcost = pathCost(abschnitt(path,0,len(path)-1))

    best = maxcost
    versuch = maxcost//2

    while True:

        if aufteilungOK(path,versuch):
            best = versuch
            versuch = versuch//2
        else:
            versuch1 = (versuch+best) // 2
            if versuch1 == versuch: 
                 break
            versuch = versuch1
            
    return best, aufteilung(path,best)


nr = 2
inf = float('inf')
f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
anzV, anzE = [int(x) for x in f.readline().split()]

GM = [[inf]*anzV for _ in range(anzV)]
V = set()
E = set()
for i in range(anzE):
    von, bis, km = [int(x) for x in f.readline().split()]
    GM[von][bis]=km
    GM[bis][von]=km
    V.add(von)
    V.add(bis)
    
    if von < bis:
        E.add((von,bis))
    else:
        E.add((bis,von))
for i in range(anzV):
    GM[i][i]=0
    
D, P = floyd(GM)

f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
anzV, anzE = [int(x) for x in f.readline().split()]
G = {x:set() for x in range(anzV)}
for _ in range(anzE):
    v1, v2, _ = f.readline().split()
    v1, v2 = int(v1), int(v2)
    G[v1].add(v2)
    G[v2].add(v1)
    
U = []
for v in G:
    if len(G[v]) % 2 != 0:
        U.append(v)
        
hilf = 10000
addMeta(U)
GE = deepcopy(G)
degrees = {x:len(GE[x]) for x in GE} 
path = []
dfs(0)                         # beim undirected eulercircle unnötig
path = eliminateMeta(path)
kosten, teile =   besteAufteilung(path)

print(f'Fahrplan für Beispiel {nr}:')
maxcost = 0
for teil in teile:
    p = abschnitt(path,teil[0],teil[1])
    cost = pathCost(p)
    if cost > maxcost:
        maxcost = cost
    print(f'Kosten: {cost} - {p}')
print(f'Maximale Kosten einer Fahrt: {maxcost}')

Fahrplan für Beispiel 2:
Kosten: 11 - (0, 6, 14, 13, 9, 12, 8, 14, 7, 11, 5, 0)
Kosten: 11 - (0, 5, 11, 8, 7, 11, 5, 14, 6, 4, 6, 0)
Kosten: 11 - (0, 6, 4, 3, 11, 2, 14, 10, 4, 13, 9, 0)
Kosten: 12 - (0, 9, 13, 3, 13, 1, 7, 9, 10, 2, 10, 9, 0)
Kosten: 12 - (0, 9, 10, 2, 8, 12, 1, 6, 9, 0, 5, 9, 0)
Maximale Kosten einer Fahrt: 12
